# Finding Your Sign on the HC10 World Download

So you donated to [Gamers Outreach](https://gamersoutreach.org/), or were part of [Joe Hills' Patreon](https://www.patreon.com/c/joehills/membership) and now that the world download is out, you want to pay a visit to your sign. How do you know where to go?

In [1]:
import json
import re
from collections.abc import Collection
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
from os import environ
from pathlib import Path
from typing import Any

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size /= 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- .git (folder)
- .gitignore (1.0 B)
- .gsb_manifest (110.0 B)
- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- audio_player_data (folder)
- carpet-fixes.conf (22.0 B)
- carpet.conf (57.0 B)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (9.0 KB)
- level.dat (3.3 KB)
- level.dat_old (3.3 KB)
- playerdata (folder)
- poi (folder)
- region (folder)
- resources.zip (34.0 MB)
- scripts (folder)
- session.lock (3.0 B)
- stats (folder)


## Find a Sign

Signs are block entities. Let's have a look at one at random.

In [5]:
all_overworld_regions = sorted(
    (save_folder / "region").glob("*"), key=lambda path: -path.stat().st_size
)
all_nether_regions = sorted(
    (save_folder / "DIM-1" / "region").glob("*"), key=lambda path: -path.stat().st_size
)
all_end_regions = sorted(
    (save_folder / "DIM1" / "region").glob("*"), key=lambda path: -path.stat().st_size
)
all_region_files = all_overworld_regions + all_nether_regions + all_end_regions
for path in all_region_files[:10]:
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")
print(f"... {len(all_region_files) - 10} more")

- r.2.-1.mca (13.3 MB)
- r.-2.-1.mca (13.3 MB)
- r.-1.-1.mca (13.3 MB)
- r.-2.0.mca (12.6 MB)
- r.-4.-1.mca (12.2 MB)
- r.0.0.mca (12.1 MB)
- r.0.-6.mca (12.0 MB)
- r.-3.-1.mca (11.7 MB)
- r.-1.0.mca (11.7 MB)
- r.-3.0.mca (11.5 MB)
... 306 more


In [6]:
%%time
for path in all_region_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value == "minecraft:sign":
                break
        else:
            continue
        break
    else:
        continue
    break
summarize_keystore(entity)


 - `z` : `-504`
 - `x` : `1235`
 - `is_waxed` : `0`
 - `id` : `"minecraft:sign"`
 - `y` : `38`
 - `front_text` : (3 items)
 - `keepPacked` : `0`
 - `components` : (0 items)
 - `back_text` : (3 items)

CPU times: user 466 ms, sys: 4.59 ms, total: 471 ms
Wall time: 473 ms


Note that the block _entity_ is just a `minecraft:sign` and doesn't reflect the various wood types. This I knew from [some modding trauma.](https://github.com/OpenBagTwo/LighterEnd/blob/eb00e5b6edce191f954a1b09de351a94a71569ee/src/main/java/io/github/openbagtwo/lighterend/registries/LighterEndBlockEntities.java)

Signs are double-sided now, so we should make sure to check both sides.

In [7]:
summarize_keystore(entity["front_text"])


 - `color` : `"black"`
 - `messages` : (4 items)
 - `has_glowing_text` : `0`

In [8]:
[line.value for line in entity["front_text"]["messages"]]

['', 'Spare', 'Armor', '']

Let's make sure that hanging signs follow the same format.

In [9]:
%%time
for path in all_region_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value == "minecraft:hanging_sign":
                break
        else:
            continue
        break
    else:
        continue
    break
summarize_keystore(entity)


 - `z` : `-506`
 - `x` : `1234`
 - `is_waxed` : `0`
 - `id` : `"minecraft:hanging_sign"`
 - `y` : `38`
 - `front_text` : (3 items)
 - `keepPacked` : `0`
 - `components` : (0 items)
 - `back_text` : (3 items)

CPU times: user 465 ms, sys: 2.81 ms, total: 468 ms
Wall time: 470 ms


In [10]:
summarize_keystore(entity["front_text"])


 - `color` : `"pink"`
 - `messages` : (4 items)
 - `has_glowing_text` : `1`

In [11]:
[line.value for line in entity["front_text"]["messages"]]

['', 'Recovery', 'Armor', '']

In [12]:
type(entity)

nbt.nbt.TAG_Compound

Cool. So we have enough to build out a data extractor.

In [13]:
def extract_text_from_sign(entity: nbt.TAG_Compound) -> list[str]:
    """Extract all text lines from a sign

    Parameters
    ----------
    entity
        The sign block entity

    Returns
    -------
    list of str
        The lines of text on that sign
    """
    lines = []
    for side in ("front_text", "back_text"):
        for line in entity[side]["messages"]:
            lines.append(line.value)
    return lines

In [14]:
def find_matching_signs(path: Path, pattern: str, case_sensitive: bool = True) -> None:
    """Find and report the locations of all signs matching the
    given regular expression pattern within the specified
    region file

    Parameters
    ----------
    path : Path
        The path of the region file to scan
    pattern : str
        The regex pattern to match
    case_sensitive : bool, optional
        Whether to match capitalization. Default is True
        (match case exactly).

    Returns
    -------
    None
        Any matches are printed to stdout
    """
    match path.parent.parent.name:
        case "DIM1":
            dim = "The End"
        case "DIM-1":
            dim = "The Nether"
        case _:
            dim = "The Overworld"
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value != "minecraft:sign":
                continue

            lines = extract_text_from_sign(entity)
            for line in lines:
                if re.match(
                    pattern, line, re.NOFLAG if case_sensitive else re.IGNORECASE
                ):
                    coordinates = "({}, {}, {})".format(
                        *[entity[coord].value for coord in "xyz"]
                    )
                    print(
                        f"Matching sign (text: \"{' '.join(lines)}\")"
                        f" found at {coordinates} in {dim}"
                    )
                    break

And so:

In [15]:
%%time
with ProcessPoolExecutor(max_workers=24) as executor:
    futures = []
    for region_file_path in all_region_files:
        futures.append(
            executor.submit(
                partial(
                    find_matching_signs, pattern=r".*barlev.*", case_sensitive=False
                ),
                region_file_path,
            )
        )
    for result in as_completed(futures):
        pass

Matching sign (text: " The Family Barlev     ") found at (72, 130, -273) in The Nether
CPU times: user 69.6 ms, sys: 66.2 ms, total: 136 ms
Wall time: 11.3 s


## A quick `/tp` later

![My sign is right where the script said it would be](../_static/sign.png)